## Import libraries

In [3]:
import src.config as config

## ETL — Extract, Transform, Load of Raw Dataset

This section is responsible for the Extraction phase of the ETL process, pulling historical financial data from multiple sources as defined in the configuration file [dataset.yaml](../configs/dataset.yaml).

- Data Storage: Each dataset is stored locally in the data/raw/ directory, organized and saved individually as .csv files for traceability and versioning.
- Data Cleaning:
  - Missing values are treated using a standard cleaning strategy.
  - Features with low variance (threshold < 0.01) are removed to reduce noise and improve modeling efficiency. In this execution, no low-variance columns were found.

In [4]:
from src import dataset

dataset.main(
    # ---- REPLACE DEFAULT AS APPROPRIATE ----
    asset = '^BVSP',
    asset_focus = 'Close',
    years = 9
    # -----------------------------------------
    )

2025-06-21 13:22:01.090 | INFO     | src.dataset:main:32 - Starting raw data loading...
2025-06-21 13:22:01.090 | INFO     | src.dataset:main:38 - Requesting information between 2015-06-24 and 2025-06-21
2025-06-21 13:22:01.109 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading BOVESPA (^BVSP) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-21 13:22:03.852 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading S&P500 (^GSPC) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-21 13:22:06.725 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading BITCOIN (BTC-USD) from yfinance...


...

2025-06-21 13:22:18.451 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC (11) from the Central Bank of Brazil API...
2025-06-21 13:22:40.480 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading CDI (12) from the Central Bank of Brazil API...
2025-06-21 13:23:03.250 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC_Anual (1178) from the Central Bank of Brazil API...
2025-06-21 13:23:25.115 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC_Meta_Anual (432) from the Central Bank of Brazil API...
2025-06-21 13:23:55.294 | WARNING  | src.utils.dataset.dataset_loading_strategy:_request_bcb_series:118 - Erro ao conectar à API do BCB: Expecting value: line 1 column 1 (char 0)
2025-06-21 13:23:55.298 | WARNING  | src.utils.dataset.dataset_loading_strategy:_load_single_ticker:94 - No data returned for SELIC_Meta_Anual (432)
2025-06-21 13:23:57.300 | INFO     | src.utils

INFO:src.utils.dataset.clean_strategy:Executing CleanMissingValues...
INFO:src.utils.dataset.clean_strategy:Executing CleanLowVariance with threshold=0.01...
INFO:src.utils.dataset.clean_strategy:Columns removed due to low variance: Index([], dtype='object')


2025-06-21 13:24:33.258 | SUCCESS  | src.dataset:main:64 - Raw data successfully loaded...


C:\Repositories\ds-lstm-ibov\src\utils\dataset\dataset_save_strategy.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  updated_df.bfill(inplace=True)


            ('Close', '^BVSP')  ('High', '^BVSP')  ('Low', '^BVSP')  \
2025-06-12            137800.0           137931.0          136175.0   
2025-06-13            137213.0           137800.0          136586.0   
2025-06-16            139256.0           139988.0          137212.0   
2025-06-17            138840.0           139497.0          138293.0   
2025-06-18            138717.0           139161.0          138443.0   
2025-06-19            138717.0           139161.0          138443.0   
2025-06-20            137116.0           138719.0          136815.0   

            ('Open', '^BVSP')  ('Volume', '^BVSP')  ('Close', '^GSPC')  \
2025-06-12           137127.0            7124600.0         6045.259766   
2025-06-13           137800.0            8628300.0         5976.970215   
2025-06-16           137212.0            7620500.0         6033.109863   
2025-06-17           139256.0            8377000.0         5982.720215   
2025-06-18           138844.0            8323400.0         59

## Feature Engineering — Time Series Preparation

This part of the pipeline is responsible for transforming the cleaned dataset into structured features suitable for training time series models. It includes the following key steps:
- Feature Generation: Constructs relevant features based on historical market data.
- Dataset Splitting: The dataset is split into training and testing sets using a consistent strategy to preserve temporal structure.
- Time Series Windowing: Converts the sequential data into overlapping windows, enabling the model to learn temporal dependencies.
- Saving Artifacts: Both training and testing sets are stored for reproducibility, along with the transformation pipelines applied during preprocessing.

In [5]:
from src import features

features.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    input_path = config.PROCESSED_DATA_DIR / "dataset.csv",
    train_dir = config.PROCESSED_DATA_DIR,
    test_dir = config.PROCESSED_DATA_DIR,
    # -----------------------------------------
)

2025-06-21 13:24:45.349 | INFO     | src.features:main:31 - Generating features from dataset...
2025-06-21 13:24:45.417 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-06-21 13:24:45.444 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...
2025-06-21 13:24:46.003 | SUCCESS  | src.features:main:46 - Saving train features in C:\Repositories\ds-lstm-ibov\data\processed...
2025-06-21 13:24:46.420 | SUCCESS  | src.features:main:50 - Saving test features in C:\Repositories\ds-lstm-ibov\data\processed...
2025-06-21 13:24:46.429 | SUCCESS  | src.features:main:54 - Features generation complete.
2025-06-21 13:24:46.429 | INFO     | src.features:main:56 - Saving transformers...
2025-06-21 13:24:46.433 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-06-21 13:24:46.447 | INFO     | src.features:main:72 - Total t

## Modeling

The modeling process begins with loading the training dataset. Next, the base model is constructed, and both the compilation and training strategies are defined. With the pipeline structure in place, the model is trained over 100 epochs using an iterative approach to adjust the weights.

During training, key metrics are monitored, including accuracy, loss (error), validation accuracy and loss (on unseen data), as well as the learning rate.

In [6]:
from src.modeling import train

train.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    X_path = config.PROCESSED_DATA_DIR / "X_train.npy",
    y_path = config.PROCESSED_DATA_DIR / "y_train.npy"
    # -----------------------------------------
)

2025-06-21 13:24:46.657 | INFO     | src.modeling.train:main:29 - Loading training dataset...
2025-06-21 13:24:46.778 | INFO     | src.modeling.train:main:37 - Building model...
2025-06-21 13:24:46.778 | INFO     | src.modeling.train:main:43 - Selecting compile strategy...
2025-06-21 13:24:46.778 | INFO     | src.modeling.train:main:46 - Selecting training strategy...
2025-06-21 13:24:46.778 | INFO     | src.modeling.train:main:49 - Building model training pipeline template...
2025-06-21 13:24:46.778 | INFO     | src.modeling.train:main:56 - Training model...
Epoch 1/1000
2025-06-21 13:25:02.888 | ERROR    | src.utils.train.train_strategy:train:33 - Error training RegressionTrainStrategy: Graph execution error:

Detected at node gradient_tape/compile_loss/huber_loss/BroadcastGradientArgs_2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Repositories\ds-lstm-ibov\.venv\Lib\site-pack

AttributeError: 'Sequential' object has no attribute 'epoch'

## Predict

In the prediction stage, the trained model is loaded along with the input data. The model then performs inference, generating predictions based on the provided data.


In [ ]:
from src.modeling import predict
predict.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    input_path = config.PROCESSED_DATA_DIR / "dataset.csv",
    preprocessor_path = config.PROCESSED_DATA_DIR / "preprocessor.pkl",
    model_path = config.MODELS_DIR / "Sequential_epoch137_loss0.0081.keras",  # Select the best model  
    postprocessor_path = config.PROCESSED_DATA_DIR / "postprocessor.pkl",
    output_path = config.PROCESSED_DATA_DIR / "dataset_report.csv",
    # -----------------------------------------
)

## Plot

Open the [Power BI Report]((../reports/pbi/amp-fynance.pbip)) and refresh data.